In [1]:
!pip install rouge
!pip install transformers
!pip install tqdm
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 76.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 101.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 28.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import json, gzip
from transformers import BartForConditionalGeneration, BartTokenizer
from rouge import Rouge
import random
from tqdm import tqdm
import nltk
from nltk.translate import meteor_score
import string
nltk.download('wordnet')
nltk.download('wordnet_ic')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package wordnet_ic to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet_ic.zip.


True

In [3]:
# model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")
# tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")
model = BartForConditionalGeneration.from_pretrained("sshleifer/distilbart-cnn-12-6")
tokenizer = BartTokenizer.from_pretrained("sshleifer/distilbart-cnn-12-6")
rouge = Rouge()

In [8]:
path = "/content/sci_articles_50.txt"
articles = []
f = open(path, "r")
text = ""
for ln in f:
    if "\n" in ln:
      text += ln[:-2]
      articles.append(text)
      text = ""


In [11]:
path = "/content/sci_sums_50.txt"
summaries = []
f = open(path, "r")
summary = ""
for ln in f:
    if "\n" in ln:
      summary += ln[:-2]
      summaries.append(summary)
      summary = ""


In [22]:
# random.shuffle(data)
numArticles = len(summaries)
totScore_l = 0
totScore_1 = 0
totScore_2 = 0
metScore = 0
# to remove punctuation
translator = str.maketrans('', '', string.punctuation)

for i in tqdm(range(len(summaries))):
    text = articles[i]
    summary = summaries[i]
    # title = article["title"]

    inputs = tokenizer.batch_encode_plus([text], max_length=1024, return_tensors="pt")
    summary_ids = model.generate(
        inputs["input_ids"], num_beams=4, max_length=200, early_stopping=True
    )
    bart_summary = tokenizer.decode(summary_ids.squeeze(), skip_special_tokens=True)

    # print()
    # print("Given Summary")
    # print(summary)
    # print("BART Summary")
    # print(bart_summary)

    # ROUGE
    scores = rouge.get_scores(bart_summary, summary)
    totScore_l += scores[0]["rouge-l"]["f"]
    totScore_1 += scores[0]["rouge-1"]["f"]
    totScore_2 += scores[0]["rouge-2"]["f"]

    # METEOR
    ref = bart_summary.translate(translator).split()
    hyp = summary.translate(translator).split()
    score = meteor_score.meteor_score([ref], hyp)
    metScore += score

print()
print("Rouge_l")
print("Total: " + str(totScore_l))
print("Average: " + str(totScore_l / numArticles))
print()
print("Rouge_1")
print("Total: " + str(totScore_1))
print("Average: " + str(totScore_1 / numArticles))
print()
print("Rouge_2")
print("Total: " + str(totScore_2))
print("Average: " + str(totScore_2 / numArticles))
print()
print("METEOR")
print("Total: " + str(metScore))
print("Average: " + str(metScore / numArticles))

100%|██████████| 49/49 [16:12<00:00, 19.85s/it]


Rouge_l
Total: 15.184722000154007
Average: 0.3098922857174287

Rouge_1
Total: 15.645159243024768
Average: 0.3192889641433626

Rouge_2
Total: 9.190306848805465
Average: 0.18755728262868296

METEOR
Total: 17.503930805127577
Average: 0.35722307765566486
